# TME 3 :  Evaluation des résultats de prédiction de gènes



In [1]:
import doctest # Pour pouvoir utiliser doctest.testmod()
import numpy as np

### A) Matrice de confusion et mesures de performance 

Il est nécessaire d’avoir une méthode pour évaluer la qualité des prédictions obtenues pour les coordonnées des gènes. Est ce que certains gènes sont ratés par la prédiction, ou est-ce qu’au contraire certains des gènes sont systématiquement surannotés alors qu’il s’agit d’une région intergénique (entre les gènes) ? Toute l’information peut être résumée en construisant le tableau de valeurs suivante, appelée matrice de confusion. Pour cela nous attribuons à chaque nucléotide d’un génome donné la valeur 0 ou 1, 0 si le nucléotide ne fait pas partie d’un gène et 1 si il en fait partie. La matrice de confusion est calculée comme ce tableau. 

![](matriceconfusion.png "")

On résume ensuite la qualité de l’annotation avec les métriques suivantes : 

   + La sensibilité  (= ``11/(11+10)``) <br>
Sen = (le taux de vrais positifs) / (le taux de vrais positifs + le taux de faux négatifs)

   + La spécificité (= ``00 / (00+01)``) <br>
Spe = (le taux de vrais négatifs) / (le taux de vrais négatifs + le taux de faux positifs)

   + La valeur prédictive VP (= ``11 / (11+01)``) est le taux de vrais positifs parmi ceux prédits comme positifs <br>
VP = (le taux de vrais positifs) / (le taux de vrais positifs + le taux de faux positifs)

<br>




<b>Question 1)</b> Ecrivez une fonction ``eval_res`` qui calcule la Sen et la Spe et VP à partir d’une matrice de confusion et renvoie un tuple à trois dimensions. La matrice de confusion sera codée comme un tableau à deux dimension, _e.g._ une liste de listes ou une matrice de numpy.


In [2]:
# Question 1
def eval_res(matconf):
    """Renvoi un tuple avec la sensibilité, la spécificité et la valeur prédictive."""
    sen = matconf[1][1]/(matconf[1][1]+matconf[1][0])
    spe = matconf[0][0]/(matconf[0][0]+matconf[0][1])
    vp = matconf[1][1]/(matconf[1][1]+matconf[0][1])
    return (sen, spe, vp)

In [3]:
assert eval_res([[50, 10], [5, 100]]) == (100 / 105, 50 / 60, 100 / 110)

<b>Question 2)</b> Ecrivez une fonction ``ecrit_intervalle`` qui, à partir des deux listes des positions de début et de fin des gènes sur le brin positif et de la longueur du génome, recode une grande liste avec la valeur 1 pour les positions qui sont des gènes et 0 sinon. Attention, il faut tenir compte des cas possibles où des gènes se chevauchent. Vous pourrez par exemple initialiser une liste avec ``lg`` valeurs à ``0`` et traiter ensuite les intervalles séquentiellement.


In [4]:
# Question 2
def ecrit_intervalle(positions_debut, positions_fin, len_genome):
    """
    Renvoie une liste binaire avec un élément par base de génome.
    
    1 indique la présence d'un gène.
    
    >>> ecrit_intervalle([2, 10, 15], [7, 12, 20], 22)
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0]
    """
    posdeb=positions_debut
    posfin=positions_fin
    genome_list=[]
    i=0
    while (i<len_genome):
        genome_list.append(0)
        i+=1
    i=0
    while (i<len_genome):
        if (posdeb != [] and posfin !=[]):
            if (i>=posdeb[0] and i<=posfin[0]):
                genome_list[i]=1
            if (i>posdeb[0] and i>posfin[0]):
                posdeb.remove(posdeb[0])
                posfin.remove(posfin[0])
        i+=1
    return genome_list


doctest.testmod()

TestResults(failed=0, attempted=1)

**Question 3)** Ecrivez la fonction ``compare_intervalle`` qui compare deux listes produites par la fonction ``ecrit_intervalle``, et renvoie la matrice de confusion.  
    Par exemple, si les deux listes données en paramètre sont  
    ``génome =[0,0,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0]`` et  
    ``ORFs   =[0,0,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0]``, on a :  

| Genome / ORF | 0 | 1 |
|---|---|---| 
| **0** | 7 | 0  |
| **1** | 4 | 11 |


In [5]:
# Question 3
def compare_intervalles(intervalle_1, intervalle_2):
    """Renvoi la matrice de confusion."""
    matrix = [[0, 0], [0, 0]]
    i=0
    while(i<len(intervalle_1)):
        if(intervalle_1[i]==0 and intervalle_2[i]==0):
            matrix[0][0]+=1
        elif(intervalle_1[i]==0 and intervalle_2[i]==1):
            matrix[0][1]+=1
        elif(intervalle_1[i]==1 and intervalle_2[i]==0):
            matrix[1][0]+=1
        else:
            matrix[1][1]+=1
        i+=1
    return matrix

In [6]:
compare_intervalles(
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

[[7, 0], [4, 11]]

### B) Evaluation sur _E. coli_

<b>Question 1)</b> Appliquez ``compare_intervalle`` à l’annotation des gènes sur le brin positif et à la prédiction par les ORFs pour générer la matrice de confusion. Et calculez ensuite la Sen et la Spe et VP pour évaluer l’annotation. **Attention, certains gènes définit dans les fichiers .tab sont non-codant (rRNA transfer, rRNA ribossomal). Ces gènes ne commencent pas par un codon START et  nous allons les ignorer dans notre comparaison**. Donnez les résultats pour l’annotation obtenue à la question précédente pour le génome de Coli. Est ce que cela vous semble être un bon résultat ?




In [7]:
import pandas as pd
import gzip
import shutil
import urllib.request

In [8]:
urllib.request.urlretrieve(
    'ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_feature_table.txt.gz',
    'GCF_000026345.1_ASM2634v1_feature_table.txt.gz')

('GCF_000026345.1_ASM2634v1_feature_table.txt.gz',
 <email.message.Message at 0x21b11287af0>)

In [9]:
urllib.request.urlretrieve(
    'ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_genomic.fna.gz',
    'GCF_000026345.1_ASM2634v1_genomic.fna.gz')

('GCF_000026345.1_ASM2634v1_genomic.fna.gz',
 <email.message.Message at 0x21b11287a60>)

In [10]:
with gzip.open('GCF_000026345.1_ASM2634v1_feature_table.txt.gz', 'rb') as f_in:
    with open('GCF_000026345.1_ASM2634v1_feature_table.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [11]:
with gzip.open('GCF_000026345.1_ASM2634v1_genomic.fna.gz', 'rb') as f_in:
    with open('GCF_000026345.1_ASM2634v1_genomic.fna', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [33]:
#grep -v ">" GCF_000026345.1_ASM2634v1_genomic.fna > GCF_000026345.1_ASM2634v1_genomic.fna.clean

file = open("GCF_000026345.1_ASM2634v1_genomic_orf_longerthan500.fna","r")
lines = file.readlines()
file.close()
cleanfile = open("GCF_000026345.1_ASM2634v1_genomic.fna.clean","w")
for line in lines:
    if not line.startswith(">"):
        cleanfile.write(line)



In [13]:
#utiliser Pandas pour lire le fichier feature_table.txt
df = pd.read_csv('GCF_000026345.1_ASM2634v1_feature_table.txt')
fichier = open('testpanda.txt',w)
fichier.write(df)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 187, saw 2


In [34]:
lisdeb=[]
lisfin=[]

with open("GCF_000026345.1_ASM2634v1_feature_table.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith("gene"):
            start = int(line.strip().split("\t")[7])
            end = int(line.strip().split("\t")[8])
            lisdeb.append(start)
            lisfin.append(end)

cleanfile = open("GCF_000026345.1_ASM2634v1_genomic.fna.clean","r")
txt=cleanfile.read().replace('\n','')
annotation=ecrit_intervalle(lisdeb,lisfin,len(txt))

listdeb=[]
listfin=[]

with open("tab_genomic_ORF.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        if not line.startswith("#"):
            start = int(line.strip().split("\t")[0])
            end = int(line.strip().split("\t")[1])
            listdeb.append(start)
            listfin.append(end)

prediction=ecrit_intervalle(listdeb,listfin,len(txt))

M=compare_intervalles(annotation,prediction)
print(M)
print(eval_res(M))
#fichier = open("tab_genomic_ORF.txt", "w")

[[196977, 6511], [966524, 725271]]
(0.4286991036147997, 0.9680030272055354, 0.9911025414672676)


<b>Question 2)</b> Analyse des résultats. Pour comprendre les limitations d'une méthode de prédiction on doit toujours analyser dans un second temps les erreurs faites par le programme. En détectant des erreurs récurrente, il est plus facile de proposer ensuite des améliorations. Donnez deux exemples de régions d'au moins 50 nucléotides consécutifs qui sont :
    - un faux négatif (gène raté)
    - un faux positif (gène inexistant)
A votre avis à quoi ces deux types d'erreurs sont-elles dues ? 

In [32]:
# Les valeurs suivantes ont pu etre observées grâce à la création d'un tableau regroupant les positions des codons start et stop des gens presents dans l'annotation
# Nous ne considérons que les ORF contenant au minimum 500 nucleotides
#
# FAUX NEGATIF : START = 5232,    STOP = 5528     on l'observe dans la prediction mais pas dans notre annotation a cause de notre limite basse de 500 nucleotides
#                START = 23091,   STOP = 25541    on l'observe dans la prediction mais pas dans notre annotation 
#
# FAUX POSITIF : START = 67020,   STOP = 67551    on l'observe dans l'annotation
#                START = 67802,   STOP = 68378    on l'observe ce gene dans l'annotation
#                START = 67030,   STOP = 68378    on l'observe ce gene dans la prediction
#                On observe que dans notre annotation, nous notons deux genes alors que dans la prediction il n'y en a qu'un
# 
# Notre valeur basse de sensibilitée peut s'expliquer par le fait que si deux gènes sont imbriquées, on n'en relèvera qu'un seul ce qui augmente notre nombre de vrais
# négatifs mais fait aussi augmenter notre nombre de faux negatifs : nous ne detectons pas les genes imbriquées.


In [31]:
# tests personnels

lisdeb=[]
lisfin=[]
with open("GCF_000026345.1_ASM2634v1_feature_table.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith("gene"):
            start = int(line.strip().split("\t")[7])
            end = int(line.strip().split("\t")[8])
            lisdeb.append(start)
            lisfin.append(end)

# recherche du plus petit gene dans la prediction
min = 10000
i = 0
liste_len_gene = []
while i < len(lisdeb):
    len_gene = lisfin[i] - lisdeb[i]
    liste_len_gene.append(len_gene)
    if len_gene < min:
        min = len_gene
    i += 1
print("min =",min)
print("len liste =",len(liste_len_gene))

# on regarde la taille des genes 
cpt = 0
for i in liste_len_gene:
     if i < 500:
         cpt += 1
print("cpt =",cpt)

min = 41
len liste = 4948
cpt = 1352
